# 🤖 Ders 3 Uygulaması: Dinamik Veri Çoğaltma (Data Augmentation) Aracı

Bu proje, Ders 3 notlarında detaylandırılan "Veri Çoğaltma" (Data Augmentation) tekniklerini interaktif bir Python aracı haline getirmektedir. Amaç, kısıtlı veri seti problemini aşmak, modelin ezberleme (overfitting) yapmasını engellemek ve genelleştirme (generalization) yeteneğini güçlendirmektir.

### Dosya Yolu (File Path) Entegrasyonu
Bu simülasyon, makine öğrenmesi projelerinde veri setlerinin bilgisayardan nasıl okunduğunu anlatan temel bir I/O (Girdi/Çıktı) mimarisine sahiptir:
* **Göreceli (Relative) Yol:** Betik ile aynı klasördeki dosyaları (örn: `araba.jpg`) okuma yeteneği.
* **Mutlak (Absolute) Yol:** Bilgisayarın farklı dizinlerindeki dosyaları (örn: `C:\Resimler\araba.jpg`) okuma yeteneği.
* **Hata Yönetimi (Try-Except):** Yanlış dosya yolu girildiğinde sistemin çökmesini engelleyip otomatik olarak sentetik (yedek) verilere geçiş yapan hata yakalama mekanizması.

### Sistem Tarafından Üretilen Sentetik Veriler
Kullanıcı dışarıdan bir veri beslemediğinde, sistem eğitim için iki farklı formda sentetik veri üretebilir:
1.  **Dur Tabelası:** Geometrik, köşeli ve simetrik bir yapı.
2.  **Ağaç:** Doğal, yuvarlak hatlı ve asimetrik bir yapı.

### Dönüşüm Kuralları ($x_{yeni} = T(x)$)
Orijinal veri ($x$) ister kullanıcının yüklediği bir kedi fotoğrafı olsun, ister sistemin çizdiği bir ağaç olsun; kod bu veriye çeşitli matematiksel transformasyonlar ($T$) uygular. Önemli olan kural, veri görsel olarak bozulsa bile, yapay zekaya öğretilen hedefin/etiketin ($y$) algoritma tarafında sabit tutulmasıdır. 

Uygulanan standart dönüşümler; Döndürme (Rotation), Çevirme (Flipping), Parlaklık Değişimi (Brightness) ve Gürültü Ekleme (Noise) işlemlerini kapsar.

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageEnhance

# --- 1. ADIM: VERI YÜKLEME VE KULLANICI ETKILEŞIMI ---
def gorsel_yukle_veya_yarat():
    print("-" * 60)
    print("VERI ÇOĞALTMA (DATA AUGMENTATION) ARACI")
    print("-" * 60)
    print("Lütfen bir veri kaynağı seçin:")
    print("1 - Kendi dosyamı yüklemek istiyorum (Dosya Yolu ile)")
    print("2 - Sistem Örneği 1: Dur Tabelası (Geometrik ve Köşeli)")
    print("3 - Sistem Örneği 2: Ağaç (Doğal ve Yuvarlak Hatlı)")
    
    secim = input("\nSeçiminiz (1/2/3): ")

    if secim == '1':
        print("\nBILGI: Dosyanız kod ile aynı klasördeyse sadece adını (ornek.jpg) yazın.")
        print("Farklı bir klasördeyse tam adresini (C:\\Masaustu\\ornek.jpg) yazın.")
        dosya_yolu = input("Dosya yolunu girin: ")
        
        # Dosyanın var olup olmadığını ve okunabilirliğini kontrol ediyoruz
        if os.path.exists(dosya_yolu):
            try:
                img = Image.open(dosya_yolu).convert('RGB')
                img = img.resize((300, 300)) 
                print(f"BAŞARILI: '{dosya_yolu}' sisteme yüklendi.")
                return img
            except Exception as e:
                print(f"HATA: Dosya okunamadı ({e}). Varsayılan 2. örneğe geçiliyor.")
        else:
            print("HATA: Belirttiğiniz yolda bir dosya bulunamadı! Varsayılan 2. örneğe geçiliyor.")
            
    elif secim == '3':
        # Alternatif Örnek: Ağaç Çizimi
        print("BILGI: 'Ağaç' verisi sentetik olarak oluşturuluyor.")
        img = Image.new('RGB', (300, 300), color='lightcyan')
        draw = ImageDraw.Draw(img)
        # Ağaç gövdesi (Kahverengi dikdörtgen)
        draw.rectangle([(130, 150), (170, 300)], fill='saddlebrown')
        # Ağaç yaprakları (Yeşil elips)
        draw.ellipse([(50, 20), (250, 200)], fill='forestgreen')
        return img

    # Varsayılan / Hata Durumu veya Seçim 2: Dur Tabelası
    print("BILGI: 'Dur Tabelası' verisi sentetik olarak oluşturuluyor.")
    img = Image.new('RGB', (300, 300), color='skyblue')
    draw = ImageDraw.Draw(img)
    draw.polygon([(90, 30), (210, 30), (270, 90), (270, 210), (210, 270), (90, 270), (30, 210), (30, 90)], fill='red')
    draw.text((120, 140), "DUR", fill='white')
    return img

orijinal_veri = gorsel_yukle_veya_yarat()

# --- 2. ADIM: DÖNÜŞÜM (TRANSFORMATION - T) FONKSIYONLARI ---

def dondur(image, derece):
    return image.rotate(derece, expand=False, fillcolor='black')

def cevir_yatay(image):
    return image.transpose(Image.FLIP_LEFT_RIGHT)

def parlaklik_degistir(image, oran):
    enhancer = ImageEnhance.Brightness(image)
    return enhancer.enhance(oran)

def gurultu_ekle(image):
    img_array = np.array(image)
    noise = np.random.randint(0, 60, img_array.shape, dtype='uint8')
    noisy_img_array = np.clip(img_array + noise, 0, 255).astype('uint8')
    return Image.fromarray(noisy_img_array)

# --- 3. ADIM: VERI ÇOĞALTMA IŞLEMI (PIPELINE) ---
veriler = [
    ("Orijinal (Net Veri)", orijinal_veri),
    ("Döndürülmüş (-20 Derece)", dondur(orijinal_veri, -20)),
    ("Çevrilmiş (Aynalama)", cevir_yatay(orijinal_veri)),
    ("Karanlık (Kapalı Hava)", parlaklik_degistir(orijinal_veri, 0.3)),
    ("Gürültülü (Kamera Tozu)", gurultu_ekle(orijinal_veri))
]

# --- 4. ADIM: SONUÇLARI GÖRSELLEŞTİRME ---
print("\nMatematiksel dönüşümler (T) uygulandı. Görüntüler ekrana yansıtılıyor...")

fig, axes = plt.subplots(1, 5, figsize=(16, 4))
fig.canvas.manager.set_window_title("Makine Öğrenmesi: Veri Çoğaltma Aracı")

for ax, (baslik, img) in zip(axes, veriler):
    ax.imshow(img)
    ax.set_title(baslik, fontsize=10, pad=10, fontweight='bold')
    ax.axis('off')

plt.tight_layout()
plt.show()